In [49]:
import warnings
warnings.filterwarnings('ignore')

# Import dependencies
import pandas as pd
import numpy as np
from config import db_password
from sqlalchemy import create_engine
import psycopg2
import pandas.io.sql as sqlio
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

In [50]:
#create database connection variable 
conn = psycopg2.connect(user="postgres", password=db_password, host="localhost", database="lichess_data")

In [51]:
#execute query and save it to a variable
query="select * from chess_data"
chess_df = sqlio.read_sql_query(query,conn)

In [52]:
chess_df.drop(['id'],axis=1, inplace=True)

In [53]:
# Split moves column into moves df
moves_df = chess_df['moves'].str.split(' ', n=10, expand=True)

# Drop column 10 and rename columns
moves_df=moves_df.drop(10,axis=1)
moves_df.columns= ["Wm1","Bm1","Wm2","Bm2","Wm3","Bm3","Wm4","Bm4","Wm5","Bm5"]

moves_df["outcome"] = chess_df["winner"]

# drop na
moves_df = moves_df.dropna()

In [54]:
# Changing moves to numbers
for col in moves_df.columns:
    print(col,end=' ')
    
    # Get list of unique values
    values = list(set(moves_df[col].values))
    
    # Create numerical dictionary
    values_with_indexes = {}
    for i, v in enumerate(values):
        values_with_indexes[v] = i
    
    # Replace column
    moves_df.replace({col: values_with_indexes},inplace=True)

moves_df.head()

Wm1 Bm1 Wm2 Bm2 Wm3 Bm3 Wm4 Bm4 Wm5 Bm5 outcome 

,Wm1,Bm1,Wm2,Bm2,Wm3,Bm3,Wm4,Bm4,Wm5,Bm5,outcome
0,16,5,94,124,208,138,64,120,44,161,1
1,16,13,70,106,221,292,23,105,119,14,0
2,3,8,56,17,21,76,42,411,73,267,1
3,16,5,28,79,84,133,255,454,226,118,1
4,3,8,28,17,212,68,138,4,380,609,1


In [55]:
# Export csv containing sample data to be imported into ML
# moves_df.to_csv("ML_sample_data.csv")

In [56]:
# Split our preprocessed data into our features and target arrays
y = moves_df["outcome"].values
X = moves_df.drop("outcome",1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 1)
number_input_features = len(X_train[0])

## Model 1: sigmoid input, sigmoid output

In [81]:
# Create the keras sequential model
nn_model = tf.keras.models.Sequential()

# Add the first layer including input layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="sigmoid", input_dim = number_input_features))

# Add the ouput layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Create a summary to check the structure of the sequential model
nn_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 6         
Total params: 61
Trainable params: 61
Non-trainable params: 0
_________________________________________________________________


In [82]:
# Compile the model and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [83]:
# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=5)

Epoch 1/5
754604/754604 [==============================] - 10s 14us/sample - loss: 0.6836 - acc: 0.4972
Epoch 2/5
754604/754604 [==============================] - 10s 13us/sample - loss: 0.6821 - acc: 0.4976
Epoch 3/5
754604/754604 [==============================] - 10s 14us/sample - loss: 0.6821 - acc: 0.4976
Epoch 4/5
754604/754604 [==============================] - 10s 13us/sample - loss: 0.6821 - acc: 0.4976
Epoch 5/5
754604/754604 [==============================] - 10s 13us/sample - loss: 0.6821 - acc: 0.4976


In [84]:
# now that our deep learning model is properly trained, we can evaluate the model's performance by testing its
# predictive capabilities on our testing dataset
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

251535/251535 - 2s - loss: 0.6819 - acc: 0.4983
Loss: 0.6818634756792873, Accuracy: 0.4983481466770172


## Model 2: relu input, sigmoid output

In [85]:
# Create the keras sequential model
nn_model = tf.keras.models.Sequential()

# Add the first layer including input layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim = number_input_features))

# Add the ouput layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Create a summary to check the structure of the sequential model
nn_model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 6         
Total params: 61
Trainable params: 61
Non-trainable params: 0
_________________________________________________________________


In [86]:
# Compile the model and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [87]:
# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=5)

Epoch 1/5
754604/754604 [==============================] - 10s 14us/sample - loss: 0.9443 - acc: 0.4878
Epoch 2/5
754604/754604 [==============================] - 10s 14us/sample - loss: 0.7437 - acc: 0.4869
Epoch 3/5
754604/754604 [==============================] - 10s 14us/sample - loss: 0.7417 - acc: 0.4884
Epoch 4/5
754604/754604 [==============================] - 10s 14us/sample - loss: 0.7347 - acc: 0.4879
Epoch 5/5
754604/754604 [==============================] - 10s 14us/sample - loss: 0.7300 - acc: 0.4890


In [88]:
# now that our deep learning model is properly trained, we can evaluate the model's performance by testing its
# predictive capabilities on our testing dataset
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

251535/251535 - 2s - loss: 0.6922 - acc: 0.4983
Loss: 0.6922125508361225, Accuracy: 0.49832427501678467


## Model 3: tanh input, sigmoid output

In [89]:
# Create the keras sequential model
nn_model = tf.keras.models.Sequential()

# Add the first layer including input layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="tanh", input_dim = number_input_features))

# Add the ouput layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Create a summary to check the structure of the sequential model
nn_model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 6         
Total params: 61
Trainable params: 61
Non-trainable params: 0
_________________________________________________________________


In [90]:
# Compile the model and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [91]:
# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=5)

Epoch 1/5
754604/754604 [==============================] - 10s 13us/sample - loss: 0.6830 - acc: 0.4972
Epoch 2/5
754604/754604 [==============================] - 11s 14us/sample - loss: 0.6821 - acc: 0.4976
Epoch 3/5
754604/754604 [==============================] - 10s 14us/sample - loss: 0.6820 - acc: 0.4976
Epoch 4/5
754604/754604 [==============================] - 10s 13us/sample - loss: 0.6820 - acc: 0.4976
Epoch 5/5
754604/754604 [==============================] - 10s 14us/sample - loss: 0.6820 - acc: 0.4976


In [92]:
# now that our deep learning model is properly trained, we can evaluate the model's performance by testing its
# predictive capabilities on our testing dataset
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

251535/251535 - 2s - loss: 0.6819 - acc: 0.4983
Loss: 0.6818692683180706, Accuracy: 0.4983481466770172


## Model 4: sigmoid input, linear output

In [93]:
# Create the keras sequential model
nn_model = tf.keras.models.Sequential()

# Add the first layer including input layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="sigmoid", input_dim = number_input_features))

# Add the ouput layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Create a summary to check the structure of the sequential model
nn_model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 6         
Total params: 61
Trainable params: 61
Non-trainable params: 0
_________________________________________________________________


In [94]:
# Compile the model and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [95]:
# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=5)

Epoch 1/5
754604/754604 [==============================] - 11s 15us/sample - loss: 0.6861 - acc: 0.4974
Epoch 2/5
754604/754604 [==============================] - 11s 15us/sample - loss: 0.6824 - acc: 0.4976
Epoch 3/5
754604/754604 [==============================] - 10s 14us/sample - loss: 0.6824 - acc: 0.4976
Epoch 4/5
754604/754604 [==============================] - 10s 14us/sample - loss: 0.6828 - acc: 0.4976
Epoch 5/5
754604/754604 [==============================] - 11s 14us/sample - loss: 0.6834 - acc: 0.4976


In [96]:
# now that our deep learning model is properly trained, we can evaluate the model's performance by testing its
# predictive capabilities on our testing dataset
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

251535/251535 - 2s - loss: 0.6826 - acc: 0.4983
Loss: 0.6825688758876884, Accuracy: 0.4983282685279846


## Model 5: relu input, linear output

In [97]:
# Create the keras sequential model
nn_model = tf.keras.models.Sequential()

# Add the first layer including input layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim = number_input_features))

# Add the ouput layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Create a summary to check the structure of the sequential model
nn_model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 6         
Total params: 61
Trainable params: 61
Non-trainable params: 0
_________________________________________________________________


In [98]:
# Compile the model and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [99]:
# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=5)

Epoch 1/5
754604/754604 [==============================] - 11s 14us/sample - loss: 7.0347 - acc: 0.4899
Epoch 2/5
754604/754604 [==============================] - 11s 15us/sample - loss: 6.5239 - acc: 0.4976
Epoch 3/5
754604/754604 [==============================] - 11s 15us/sample - loss: 6.5240 - acc: 0.4976
Epoch 4/5
754604/754604 [==============================] - 11s 15us/sample - loss: 6.5240 - acc: 0.4976
Epoch 5/5
754604/754604 [==============================] - 11s 15us/sample - loss: 6.5240 - acc: 0.4976


In [100]:
# now that our deep learning model is properly trained, we can evaluate the model's performance by testing its
# predictive capabilities on our testing dataset
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

251535/251535 - 2s - loss: 6.5152 - acc: 0.4983
Loss: 6.51520274151468, Accuracy: 0.49834415316581726


## Model 6: tanh input, linear output

In [101]:
# Create the keras sequential model
nn_model = tf.keras.models.Sequential()

# Add the first layer including input layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="tanh", input_dim = number_input_features))

# Add the ouput layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Create a summary to check the structure of the sequential model
nn_model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 6         
Total params: 61
Trainable params: 61
Non-trainable params: 0
_________________________________________________________________


In [102]:
# Compile the model and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [103]:
# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=5)

Epoch 1/5
754604/754604 [==============================] - 11s 15us/sample - loss: 0.9183 - acc: 0.4969
Epoch 2/5
754604/754604 [==============================] - 11s 15us/sample - loss: 0.6825 - acc: 0.4976
Epoch 3/5
754604/754604 [==============================] - 11s 15us/sample - loss: 0.6825 - acc: 0.4976
Epoch 4/5
754604/754604 [==============================] - 11s 15us/sample - loss: 0.6825 - acc: 0.4976
Epoch 5/5
754604/754604 [==============================] - 11s 15us/sample - loss: 0.6824 - acc: 0.4976


In [104]:
# now that our deep learning model is properly trained, we can evaluate the model's performance by testing its
# predictive capabilities on our testing dataset
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

251535/251535 - 2s - loss: 0.6820 - acc: 0.4983
Loss: 0.6819516978605651, Accuracy: 0.4983481466770172


Since our last submission, we have evaluated an additional 5 models for a total of 6. The loss, accuracy scores, and parameters of our 6 models are shown below:

| Model | Loss | Accuracy | Parameters |
| :---- | :--- | :------- | :--------- |
| 1 | 0.6819 | 0.4983 | Sigmoid input with 5 nodes, sigmoid output, 5 epochs |
| 2 | 0.6922 | 0.4983 | Relu input with 5 nodes, sigmoid output, 5 epochs |
| 3 | 0.6819 | 0.4983 | Tanh input with 5 nodes, sigmoid output, 5 epochs |
| 4 | 0.6826 | 0.4983 | Sigmoid input with 5 notes, linear output, 5 epochs |
| 5 | 6.5152 | 0.4983 | Relu input wiht 5 nodes, linear output, 5 epochs |
| 6 | 0.6820 | 0.4983 | Tanh input with 5 nodes, linear output, 5 epochs |

The accuracy score for all 6 models was identical at 0.4983. An accuracy score of exactly 0.5 is the probability of randomly guessing the correct winner of a given game of chess, so the accuracy score of our models may reflect the difficulty of predicting a winner from only the first 10 moves of a game. Since games typically last far longer than 10 turns and the possible combinations of moves grow increasingly complex, this is not a surprising conclusion.

Loss function, on the other hand, was not constant. The two models with relu inputs (Models 2 and 5) had the greatest loss score, indicating that the relu activation function is likely not the best choice for our model. Additionally, the models with sigmoid outputs (Models 1, 2, and 3) had lower loss functions than their counterpart models with linear outputs (Models 4, 5, and 6 respectively). This is expected, as output from a sigmoid function tends to be very close to either 0 or 1. We are posing a question with a binary answer, so a sigmoid function is the best choice for an activation function for our output layer.

Given this information, our course of action for the final steps of completing this model will be to explore more varied input layers. This will be accomplished by varying the activation function (using sigmoid and tanh, leaving relu out), number of layers, number of nodes within each layer, and number of epochs used to train the model. We hypothesize that the sigmoid activation function will be the more appropriate choice for the input layer(s) for the same reasons it is the best choice for the output layer.